In [7]:
import json
import os
import psycopg2
import paramiko
import re
# from urllib.parse import unquote
from flask import Flask, request, jsonify
import urllib.parse
from dateutil import parser as dateutil_parser
from psycopg2 import sql
import uuid
import datetime


In [8]:
data= "jsondata%5Buuid%5D=11111d1d-e0f9-4c23-9848-81bf00004256&jsondata%5Btype%5D=4&jsondata%5Btime%5D=2024-07-10 13:54:05&jsondata%5Bsourceid%5D=9990010&jsondata%5Btargetid%5D=1&jsondata%5Bstate%5D=3&jsondata%5Bfile%5D=&jsondata%5Bsourcename%5D=ilya&jsondata%5Btargetname%5D=Terminal1(1)&jsondata%5Bsourceext%5D=&jsondata%5Btargetext%5D=[{}]&jsondata%5Bextdata%5D=&jsondata%5Btaskid%5D=b6deb111-ccfe-46e7-b672-13e395551b7f&jsondata%5Bplaylistid%5D=1_ilya_ilya.mp3&jsondata%5Bishost%5D=1&jsondata%5Bsourcepanel%5D=0&jsondata%5Btargetpanel%5D=0&jsondata%5Btasktype%5D=&jsondata%5Bdeviceinfo%5D=&jsondata%5Busertaskinfo%5D=POST / HTTP/1.1"

In [ ]:
app = Flask(__name__)


def decode_and_clean(data):
    # Декодирование URL-кодированной строки
    decoded_str = urllib.parse.unquote(data)

    # Очистка строки от невалидных символов
    decoded_str = re.sub(r'[^\x20-\x7E]', '', decoded_str)

    # Регулярное выражение для извлечения сегментов с uuid
    segments = re.split(r'(?=jsondata\[uuid\]=)', decoded_str)

    result_list = []

    for segment in segments:
        if 'jsondata[uuid]=' not in segment:
            continue

        segment = re.sub(r'jsondata\[([^\]]+)\]', r'\1', segment)

        # Разделение сегмента на пары ключ=значение
        pairs = segment.split('&')

        result = {
            "uuid": "",
            "type": "",
            "time": "",
            "sourceid": "",
            "targetid": "",
            "state": "",
            "file": "",
            "sourcename": "",
            "targetname": "",
            "sourceext": "",
            "targetext": [{}],
            "extdata": "",
            "taskid": "",
            "playlistid": "",
            "ishost": "",
            "sourcepanel": "",
            "targetpanel": "",
            "tasktype": "",
            "deviceinfo": "",
            "usertaskinfo": ""
        }

        should_skip = False

        for pair in pairs:
            if '=' in pair:
                key, value = pair.split('=', 1)
                value = value.replace('+', ' ')

                if key in result:

                    if key == 'usertaskinfo':
                        # Извлечение только части строки "POST/HTTP/любое число"
                        match = re.search(r'POST\s+.*?HTTP/\d+(\.\d+)?', value)
                        if match:
                            result[key] = match.group(0)
                        else:
                            result[key] = ""

                    elif 'GET' in value:
                        # Если GET запрос, пропускаем сегмент
                        should_skip = True
                        break

                    elif key in result:
                        if key == 'targetext':
                            if value:
                                try:
                                    value = json.loads(value)
                                except json.JSONDecodeError:
                                    value = ""
                            else:
                                value = ""
                        result[key] = value

        # Если сегмент не нужно пропускать и содержит данные, добавляем в результат
        if not should_skip and result != {}:
            result_list.append(result)

    return result_list


def save_to_db(data):
    # Подключение к базе данных
    conn = psycopg2.connect(
        dbname="...",
        user="...",
        password="...",  
        host="localhost",
        port="5432"  
    )
    cursor = conn.cursor()

    # SQL-запрос для вставки данных
    insert_query = """
    INSERT INTO data_json (
        uuid, type, time, sourceid, targetid, state, file, sourcename, targetname,
        sourceext, targettext, extdata, taskid, playlistid, ishost, sourcepanel, 
        targetpanel, tasktype, deviceinfo, usertaskinfo
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for item in data:
        # Преобразование данных
            
        item['uuid'] = str(uuid.UUID(item['uuid'])) if item['uuid'] else None
        item['type'] = int(item['type']) if item['type'] else 0
        item['time'] = dateutil_parser.isoparse(item['time']) if item['time'] else None
        item['state'] = int(item['state']) if item['state'] else 0
        item['taskid'] = str(uuid.UUID(item['taskid'])) if item['taskid'] else None
        item['ishost'] = int(item['ishost']) if item['ishost'] else 0
        item['sourcepanel'] = int(item['sourcepanel']) if item['sourcepanel'] else 0
        item['targetpanel'] = int(item['targetpanel']) if item['targetpanel'] else 0

        if 'targettext' in item:
            item['targettext'] = json.dumps(item['targettext']) if isinstance(item['targettext'],list) else item['targettext']
        else:
            item['targettext'] = json.dumps([{}])

        cursor.execute(insert_query, (
            item.get('uuid'),
            item.get('type'),
            item.get('time'),
            item.get('sourceid'),
            item.get('targetid'),
            item.get('state'),
            item.get('file'),
            item.get('sourcename'),
            item.get('targetname'),
            item.get('sourceext'),
            item.get('targettext'),
            item.get('extdata'),
            item.get('taskid'),
            item.get('playlistid'),
            item.get('ishost'),
            item.get('sourcepanel'),
            item.get('targetpanel'),
            item.get('tasktype'),
            item.get('deviceinfo'),
            item.get('usertaskinfo')
        ))

    print("Данные в бд")

    # Фиксация изменений и закрытие подключения
    conn.commit()
    cursor.close()
    conn.close()


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        print(f"Raw data received: {request}")
        print(f"FFF: {request.headers}")
        if request.is_json:
            data = request.get_json()
            processed_data = decode_and_clean(data)
            save_to_db(processed_data)
            return processed_data, 200

        else:
            data = request.get_data(as_text = True)
            processed_data = decode_and_clean(data)
            print(f"Преобразованные данные: {processed_data}")
            save_to_db(processed_data)
    return "Hello, World", 200


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
